In [1]:
!pip install transformers datasets faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Mounted at /content/drive
 ahmed-permis-conduire.pdf  'Colab Notebooks'		  out
 assignement1		     CV_AHMED_HAMMAMI_EN_V6.pdf   qa_finetuned_model_final
 cat_questions.csv	    'Google Earth'		  squad_faq.json
 cat_specs.json		    'INF8245E - Copy.7z'	 'Useit Problems.gdoc'
 cat_squad.txt		     Maitrise


In [3]:
import os

cat_faq_dataset_path = "/content/drive/My Drive/squad_faq.json"
cat_specs_dataset_path = "/content/drive/My Drive/cat_specs.json"

if not os.path.exists(cat_specs_dataset_path):
    raise FileNotFoundError(f"File not found at {cat_specs_dataset_path}")

if not os.path.exists(cat_faq_dataset_path):
    raise FileNotFoundError(f"File not found at {cat_faq_dataset_path}")

In [4]:
# importing the cat_specs json which contains the parsed data from the technical doc of catnmsplan: https://www.catnmsplan.com/sites/default/files/2024-10/10.18.24_CAT_Reporting_Technical_Specifications_for_Industry_Members_v4.1.0r4_CLEAN.pdf
import json
with open(cat_specs_dataset_path, 'r') as f:
    cat_specs_data = json.load(f)

In [5]:
# importing the cat_specs json which contains the parsed data from the faq of catnmsplan: https://www.catnmsplan.com/faq
import json
with open(cat_faq_dataset_path, 'r') as f:
    cat_faq_data = json.load(f)

## Exploring question generation from cat technical spec

In [6]:
# This is just to try generating questions from the cat_specs.json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "valhalla/t5-base-qg-hl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_questions_with_t5(title, paragraph):
    context = f"{title} {paragraph}"
    highlighted_context = f"<hl> {context} <hl>"
    input_text = f"generate question: {highlighted_context}"

    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=50, num_beams=5, num_return_sequences=1)

    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return questions

i = 0
for key, value in cat_specs_data.items():
  if "content" in value:
    title = value["title"]
    paragraph = value["content"]
    questions = generate_questions_with_t5(title, paragraph)
    print("title: {}\n---------------\nparagraph: {}\n---------------\nquestions: {}\n\n\n==================================================".format(title,paragraph,questions))
    i = i + 1
    if i == 10:
      break

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

title: CAT Overview
---------------
paragraph: The Securities and Exchange Commission (SEC) approved Rule 613 under the Securities Exchange Act 
of 1934, which requires national securities exchanges and national securities associations (collectively, 
the Participants) to submit a national market system plan to create, implement, and maintain a 
consolidated audit trail (CAT NMS Plan) that would capture customer and order event information for 
orders in NMS Securities and OTC Equity Securities (Eligible Securities), across all markets, from the 
time of order inception through routing, cancellation, modification, execution, and allocation. The SEC 
approved the CAT NMS Plan on November 15, 2016. 
In accordance with SEC Rule 613, the CAT NMS Plan requires a Central Repository that will 
comprehensively track orders throughout their lifecycle and identify the Participants and Industry 
Members handling them, as well as the account holders and authorized traders for any account that 
ori

## Calculation of context from the cat technical specifciation using FAISS

In [7]:
chunks = [value["content"] for key, value in cat_specs_data.items() if "content" in value and value["content"] != ""]

In [8]:
for item in cat_faq_data['data']:
    for paragraph in item['paragraphs']:
        for qa in paragraph['qas']:
            chunks.append(qa['answers'][0]['text'])

In [9]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a model for embedding generation
seqTransformer = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
chunk_embeddings = seqTransformer.encode(chunks)

# Build FAISS index
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))

# Run one query
user_question = "What is CATnmsplan?"
query_embedding = seqTransformer.encode([user_question])
_, indices = index.search(np.array(query_embedding), k=1)  # Retrieve top 1 match
retrieved_chunk = chunks[indices[0][0]]
print(retrieved_chunk)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Information on CAT Entitlement, will be published in an Industry Member Onboarding User Guide onwww.catnmsplan.com.


## Finetuning Pretrained T5 model on CAT FAQ

In [10]:
data = []
for item in cat_faq_data['data']:
    for paragraph in item['paragraphs']:
        for qa in paragraph['qas']:
            data.append({"text": qa['answers'][0]['text'], "question": qa['question']})

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# Load pre-trained T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Convert the data to the appropriate format
def preprocess_function(examples):
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples["question"], examples["text"])]
    targets = examples["text"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Convert data into Hugging Face Dataset
dataset = Dataset.from_dict({
    "question": [d["question"] for d in data],
    "text": [d["text"] for d in data]
})

# Preprocess the data
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    fp16=True,  # Enable mixed precision training because model is too big to load in gpu memory
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_t5_model")
tokenizer.save_pretrained("./fine_tuned_t5_model")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


('./fine_tuned_t5_model/tokenizer_config.json',
 './fine_tuned_t5_model/special_tokens_map.json',
 './fine_tuned_t5_model/spiece.model',
 './fine_tuned_t5_model/added_tokens.json')

In [12]:
model.save_pretrained("/content/drive/My Drive/qa_finetuned_model_final")
tokenizer.save_pretrained("/content/drive/My Drive/qa_finetuned_model_final")

('/content/drive/My Drive/qa_finetuned_model_final/tokenizer_config.json',
 '/content/drive/My Drive/qa_finetuned_model_final/special_tokens_map.json',
 '/content/drive/My Drive/qa_finetuned_model_final/spiece.model',
 '/content/drive/My Drive/qa_finetuned_model_final/added_tokens.json')

## Using the context calculated from the question and the fine-tuned T5 model to generate a reply

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "/content/drive/My Drive/qa_finetuned_model_final"
tokenizer_loaded = T5Tokenizer.from_pretrained(model_name)
model_loaded = T5ForConditionalGeneration.from_pretrained(model_name)

In [14]:
user_question = "At what hours cat must report ?"

query_embedding = seqTransformer.encode([user_question])
_, indices = index.search(np.array(query_embedding), k=5)
retrieved_chunks = [chunks[i] for i in indices[0]]
context = " ".join(retrieved_chunks)

input_text = f"question: {user_question} context: {context}"
inputs = tokenizer_loaded(input_text, return_tensors="pt", padding=True, truncation=True)
outputs = model_loaded.generate(inputs.input_ids, max_length=512, num_beams=4, early_stopping=True)
answer = tokenizer_loaded.decode(outputs[0], skip_special_tokens=True)

print(f"Answer: {answer}")

Answer: Reports for events that occur during a particular Trading Day must be reported by 8:00 a.m., ET the following Trading Day or they are marked late by CAT. No, allocation events are required to be submitted to CAT by 8 am ET on the next “CAT Trading Day.” “CAT Trading Day” is defined as beginning immediately after 4:15:00 pm and no fractions of a second Eastern Time on one trade date and ending at exactly 4:15:00 pm and no fractions of a second Eastern Time on the next trading date. In the event that the 25th is a weekend or holiday, invoices will be published to the CAT Reporter Portal on the next CAT Trading Day, which would be by 8:00 am on Wednesday. For more information and examples of reporting deadlines, please refer to the Reporting Deadline Examples in the CAT Reporting Technical Specifications for Industry Members. For example, an allocation performed at 10 pm ET on Monday (part of Tuesday’s CAT Trading Day) would have to be reported by 8:00 am on the next CAT Trading D